In [1]:
#importing the Tensorflow and Checking the CUDA availbility
import tensorflow as tf
print(tf.test.is_built_with_cuda())


True


In [2]:
#Importing the necessary Libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,GlobalAveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,Nadam
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import DenseNet201, ResNet152V2,ResNet101V2,DenseNet121,DenseNet169,InceptionResNetV2,ResNet50V2,EfficientNetB7, NASNetLarge
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tensorflow.keras.preprocessing import image
import glob
from tensorflow.keras.metrics import Accuracy,Precision, Recall
import os
import numpy as np

In [3]:
#Maximizing the images pixels capacity
import PIL.Image
PIL.Image.MAX_IMAGE_PIXELS = None

# Data Loading

In [4]:
imagePaths = os.listdir("/media/karndeepsingh/Data-1/Data/UDC/")
os.chdir("/media/karndeepsingh/Data-1/Data/UDC/")
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
    count =0
    for img_ in glob.glob("{}/*.jpg".format(imagePath)):

        img = image.load_img(img_,target_size = (224,224))
       
        img = image.img_to_array(img)
        data.append(img)
        labels.append(imagePath)
        count +=1
    print(count)    
        


0
0
1530
1379
0
0
0
0
0
1313


In [5]:
# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)
# perform one-hot encoding on the labels#
lb = LabelEncoder()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)

# DATA AUGUMENTATION

In [6]:
data.shape

(4222, 224, 224, 3)

In [8]:
#Saving of the train and test dataset in numpy array formate into the disk
os.chdir("/media/karndeepsingh/Data-1/Data/UDC/") #saving the arrays into different location on disk
np.save("testX.npy",testX)
np.save("testY.npy",testY)
np.save("trainX.npy",trainX)
np.save("trainY.npy",trainY)

In [5]:
#Loading the saved numpy array from the disk 
os.chdir("/media/karndeepsingh/Data-1/Data/UDC/")
trainX = np.load("trainX.npy")
trainY = np.load("trainY.npy")
testX = np.load("testX.npy")
testY = np.load("testY.npy")

In [9]:
#Data Agumentation on Train Data
#trainAug = ImageDataGenerator(rotation_range=90,zoom_range =[0.5,1.0],rescale = 1./255,
#brightness_range=[0.2,1.5],
#zca_whitening =True,featurewise_center =True,featurewise_std_normalization=True,
#vertical_flip =False,
#horizontal_flip=True,
#fill_mode="nearest")

/home/codemonk-admin/anaconda3/lib/python3.8/site-packages/keras_preprocessing/image/image_data_generator.py:342: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn('This ImageDataGenerator specifies '


In [10]:
#Data Agumentation on Train Data
trainAug = ImageDataGenerator(rescale=1./255,brightness_range=[0.2,1.5])

In [11]:
#Data Augumentation on Test Data
valAug = ImageDataGenerator(rescale = 1./255)

# MODE SELECTION AND TRAINING 

In [45]:
#Loading the DenseNet MODEL
baseModel=DenseNet201(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

In [46]:
#Printing the model summary
baseModel.summary()

Model: "resnet152v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
________________________________________________________________________________________

In [47]:
#Fine tuning the Model according to our use case
flatten_images = Flatten()(baseModel.output)
headModel = Dense(512, activation="relu",kernel_initializer = "he_uniform")(flatten_images)
headModel = Dropout(0.25)(headModel)
headModel = Dense(512, activation="relu",kernel_initializer = "he_uniform")(headModel)
headModel = Dropout(0.25)(headModel)
headModel = Dense(512, activation="relu",kernel_initializer = "he_uniform")(headModel)
headModel = Dropout(0.25)(headModel)
headModel = Dense(512, activation="relu",kernel_initializer = "he_uniform")(headModel)
headModel = Dense(512, activation="relu",kernel_initializer = "he_uniform")(headModel)
headModel = Dense(64, activation="relu",kernel_initializer = "he_uniform")(headModel)
headModel = Dense(3, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_______________________________________________________________________________________

In [48]:
#Freezing the Modellayers
for layer in baseModel.layers:
    layer.trainable = False

In [49]:
#Initializing the optimisers
opt = Nadam()
model.compile(loss="categorical_crossentropy", optimizer=opt,
metrics=["accuracy","Precision","Recall"])

In [50]:
#Initializing the Earlystopping and ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=400)
mc = ModelCheckpoint('model{epoch: 03d}--{accuracy: .4f}--{val_accuracy: .4f}.h5', monito`r='val_accuracy', mode='max', verbose=1, save_best_only=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.1, verbose=1, min_lr=0)

# TRAINING OF THE MODEL

In [52]:
#TraininG
H = model.fit_generator(trainAug.flow(trainX,trainY),
steps_per_epoch=len(trainX) //128,
validation_data=valAug.flow(testX,testY),
validation_steps=len(testX) //128,
epochs=1500, callbacks = [es,mc,reduce_lr])

Epoch 1/1500
26/26 [==============================] - ETA: 0s - loss: 0.0366 - accuracy: 0.9868 - precision: 0.9892 - recall: 0.9868
Epoch 00001: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0366 - accuracy: 0.9868 - precision: 0.9892 - recall: 0.9868 - val_loss: 1.4730 - val_accuracy: 0.8333 - val_precision: 0.8413 - val_recall: 0.8281
Epoch 2/1500
26/26 [==============================] - ETA: 0s - loss: 0.0392 - accuracy: 0.9772 - precision: 0.9795 - recall: 0.9772
Epoch 00002: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0392 - accuracy: 0.9772 - precision: 0.9795 - recall: 0.9772 - val_loss: 1.3979 - val_accuracy: 0.8438 - val_precision: 0.8602 - val_recall: 0.8333
Epoch 3/1500
26/26 [==============================] - ETA: 0s - loss: 0.0371 - accuracy: 0.9868 - precision: 0.9880 - recall: 0.9856
Epoch 00003: val_accuracy did not improve from 0.91667
26/26 [===

26/26 [==============================] - ETA: 0s - loss: 0.0499 - accuracy: 0.9868 - precision: 0.9868 - recall: 0.9868
Epoch 00041: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0499 - accuracy: 0.9868 - precision: 0.9868 - recall: 0.9868 - val_loss: 1.9430 - val_accuracy: 0.8542 - val_precision: 0.8542 - val_recall: 0.8542
Epoch 42/1500
26/26 [==============================] - ETA: 0s - loss: 0.0355 - accuracy: 0.9916 - precision: 0.9940 - recall: 0.9904
Epoch 00042: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0355 - accuracy: 0.9916 - precision: 0.9940 - recall: 0.9904 - val_loss: 1.2299 - val_accuracy: 0.8490 - val_precision: 0.8571 - val_recall: 0.8438
Epoch 43/1500
26/26 [==============================] - ETA: 0s - loss: 0.0106 - accuracy: 0.9951 - precision: 0.9951 - recall: 0.9951
Epoch 00043: val_accuracy did not improve from 0.91667
26/26 [==============

26/26 [==============================] - ETA: 0s - loss: 0.0082 - accuracy: 0.9964 - precision: 0.9964 - recall: 0.9964
Epoch 00061: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0082 - accuracy: 0.9964 - precision: 0.9964 - recall: 0.9964 - val_loss: 1.5560 - val_accuracy: 0.8698 - val_precision: 0.8743 - val_recall: 0.8698
Epoch 62/1500
26/26 [==============================] - ETA: 0s - loss: 0.0253 - accuracy: 0.9964 - precision: 0.9964 - recall: 0.9964
Epoch 00062: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0253 - accuracy: 0.9964 - precision: 0.9964 - recall: 0.9964 - val_loss: 1.4203 - val_accuracy: 0.8490 - val_precision: 0.8534 - val_recall: 0.8490
Epoch 63/1500
26/26 [==============================] - ETA: 0s - loss: 0.0226 - accuracy: 0.9928 - precision: 0.9928 - recall: 0.9928
Epoch 00063: val_accuracy did not improve from 0.91667
26/26 [==============

26/26 [==============================] - ETA: 0s - loss: 0.0138 - accuracy: 0.9916 - precision: 0.9928 - recall: 0.9916
Epoch 00081: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0138 - accuracy: 0.9916 - precision: 0.9928 - recall: 0.9916 - val_loss: 1.0404 - val_accuracy: 0.8958 - val_precision: 0.9043 - val_recall: 0.8854
Epoch 82/1500
26/26 [==============================] - ETA: 0s - loss: 0.0103 - accuracy: 0.9964 - precision: 0.9964 - recall: 0.9952
Epoch 00082: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0103 - accuracy: 0.9964 - precision: 0.9964 - recall: 0.9952 - val_loss: 1.3266 - val_accuracy: 0.8698 - val_precision: 0.8743 - val_recall: 0.8698
Epoch 83/1500
26/26 [==============================] - ETA: 0s - loss: 0.0223 - accuracy: 0.9928 - precision: 0.9928 - recall: 0.9904
Epoch 00083: val_accuracy did not improve from 0.91667
26/26 [==============

26/26 [==============================] - ETA: 0s - loss: 0.0257 - accuracy: 0.9928 - precision: 0.9928 - recall: 0.9916
Epoch 00101: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0257 - accuracy: 0.9928 - precision: 0.9928 - recall: 0.9916 - val_loss: 0.7244 - val_accuracy: 0.8958 - val_precision: 0.8947 - val_recall: 0.8854
Epoch 102/1500
26/26 [==============================] - ETA: 0s - loss: 0.0143 - accuracy: 0.9951 - precision: 0.9951 - recall: 0.9951
Epoch 00102: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 121ms/step - loss: 0.0143 - accuracy: 0.9951 - precision: 0.9951 - recall: 0.9951 - val_loss: 1.2158 - val_accuracy: 0.8750 - val_precision: 0.8796 - val_recall: 0.8750
Epoch 103/1500
26/26 [==============================] - ETA: 0s - loss: 0.0221 - accuracy: 0.9928 - precision: 0.9928 - recall: 0.9928
Epoch 00103: val_accuracy did not improve from 0.91667
26/26 [============

...

Epoch 361/1500
26/26 [==============================] - ETA: 0s - loss: 0.0207 - accuracy: 0.9904 - precision: 0.9916 - recall: 0.9880
Epoch 00361: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0207 - accuracy: 0.9904 - precision: 0.9916 - recall: 0.9880 - val_loss: 1.3209 - val_accuracy: 0.8490 - val_precision: 0.8564 - val_recall: 0.8385
Epoch 362/1500
26/26 [==============================] - ETA: 0s - loss: 0.0135 - accuracy: 0.9952 - precision: 0.9952 - recall: 0.9940
Epoch 00362: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0135 - accuracy: 0.9952 - precision: 0.9952 - recall: 0.9940 - val_loss: 1.4457 - val_accuracy: 0.8281 - val_precision: 0.8307 - val_recall: 0.8177
Epoch 363/1500
26/26 [==============================] - ETA: 0s - loss: 0.0124 - accuracy: 0.9951 - precision: 0.9951 - recall: 0.9951
Epoch 00363: val_accuracy did not improve from 0.91667
26/2

Epoch 381/1500
26/26 [==============================] - ETA: 0s - loss: 0.0205 - accuracy: 0.9928 - precision: 0.9928 - recall: 0.9928
Epoch 00381: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0205 - accuracy: 0.9928 - precision: 0.9928 - recall: 0.9928 - val_loss: 1.7666 - val_accuracy: 0.8385 - val_precision: 0.8556 - val_recall: 0.8333
Epoch 382/1500
26/26 [==============================] - ETA: 0s - loss: 0.0261 - accuracy: 0.9927 - precision: 0.9926 - recall: 0.9902
Epoch 00382: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 121ms/step - loss: 0.0261 - accuracy: 0.9927 - precision: 0.9926 - recall: 0.9902 - val_loss: 1.0507 - val_accuracy: 0.8698 - val_precision: 0.8789 - val_recall: 0.8698
Epoch 383/1500
26/26 [==============================] - ETA: 0s - loss: 0.0116 - accuracy: 0.9963 - precision: 0.9963 - recall: 0.9963
Epoch 00383: val_accuracy did not improve from 0.91667
26/2

Epoch 401/1500
26/26 [==============================] - ETA: 0s - loss: 0.0182 - accuracy: 0.9916 - precision: 0.9928 - recall: 0.9916
Epoch 00401: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0182 - accuracy: 0.9916 - precision: 0.9928 - recall: 0.9916 - val_loss: 1.3153 - val_accuracy: 0.8750 - val_precision: 0.8737 - val_recall: 0.8646
Epoch 402/1500
26/26 [==============================] - ETA: 0s - loss: 0.0353 - accuracy: 0.9951 - precision: 0.9951 - recall: 0.9939
Epoch 00402: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 121ms/step - loss: 0.0353 - accuracy: 0.9951 - precision: 0.9951 - recall: 0.9939 - val_loss: 1.3480 - val_accuracy: 0.8333 - val_precision: 0.8421 - val_recall: 0.8333
Epoch 403/1500
26/26 [==============================] - ETA: 0s - loss: 0.0137 - accuracy: 0.9928 - precision: 0.9940 - recall: 0.9928
Epoch 00403: val_accuracy did not improve from 0.91667
26/2

Epoch 421/1500
26/26 [==============================] - ETA: 0s - loss: 0.0127 - accuracy: 0.9940 - precision: 0.9940 - recall: 0.9940
Epoch 00421: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0127 - accuracy: 0.9940 - precision: 0.9940 - recall: 0.9940 - val_loss: 0.6999 - val_accuracy: 0.8854 - val_precision: 0.8895 - val_recall: 0.8802
Epoch 422/1500
26/26 [==============================] - ETA: 0s - loss: 0.0342 - accuracy: 0.9904 - precision: 0.9904 - recall: 0.9892
Epoch 00422: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0342 - accuracy: 0.9904 - precision: 0.9904 - recall: 0.9892 - val_loss: 1.3649 - val_accuracy: 0.8490 - val_precision: 0.8579 - val_recall: 0.8490
Epoch 423/1500
26/26 [==============================] - ETA: 0s - loss: 0.0151 - accuracy: 0.9916 - precision: 0.9928 - recall: 0.9904
Epoch 00423: val_accuracy did not improve from 0.91667
26/2

Epoch 441/1500
26/26 [==============================] - ETA: 0s - loss: 0.0128 - accuracy: 0.9952 - precision: 0.9952 - recall: 0.9952
Epoch 00441: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0128 - accuracy: 0.9952 - precision: 0.9952 - recall: 0.9952 - val_loss: 0.8292 - val_accuracy: 0.9010 - val_precision: 0.9101 - val_recall: 0.8958
Epoch 442/1500
26/26 [==============================] - ETA: 0s - loss: 0.0146 - accuracy: 0.9904 - precision: 0.9916 - recall: 0.9904
Epoch 00442: val_accuracy did not improve from 0.91667
26/26 [==============================] - 3s 122ms/step - loss: 0.0146 - accuracy: 0.9904 - precision: 0.9916 - recall: 0.9904 - val_loss: 0.9986 - val_accuracy: 0.8646 - val_precision: 0.8684 - val_recall: 0.8594
Epoch 443/1500
26/26 [==============================] - ETA: 0s - loss: 0.0121 - accuracy: 0.9952 - precision: 0.9952 - recall: 0.9952
Epoch 00443: val_accuracy did not improve from 0.91667
26/2

# Loading Best Saved Model


In [7]:
from tensorflow.keras.models import load_model
best_model= load_model("model 09-- 0.9976-- 0.9583.h5")


# PREDICTION

In [8]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import matplotlib.pyplot as plt

In [9]:
class_labels = ["tdp", "tds","unknown"]

In [12]:
imagePaths ="/media/karndeepsingh/Data-1/Data/UDC/tds.jpg"
#os.chdir("/media/karndeepsingh/Data-1/Data/UDC/tds/")
def prediction(filepath,target_size = (224,224,3),model=best_model):   
    # Loading and resizing image
    image = load_img(filepath, target_size=target_size)
    # Convert the image pixels to a numpy array
    image = img_to_array(image)
    # Reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # Prepare the image for the CNN Model model
    image = image / 255.0
    # Pass image into model to get encoded features
    preds= model.predict(image, verbose=0)

    pred_proba =preds[0][np.argmax(preds)]
    class_l = class_labels[np.argmax(preds)]

    return class_l,pred_proba
        

In [13]:
prediction(imagePaths)

('tds', 0.9997179)

In [15]:
best_model.evaluate(trainX/255.0,trainY,steps=len(trainY)/256)

14/13 [===============================] - 6s 421ms/step - loss: 0.3720 - accuracy: 0.9671 - precision_8: 0.9683 - recall_8: 0.9665


[0.37198442220687866,
 0.9671306014060974,
 0.9682586789131165,
 0.9665383696556091]

In [21]:
predIdxs = best_model.predict(x=testX/255.0)
print(classification_report(testY.argmax(axis=1),predIdxs.argmax(axis=1), target_names = ["tdp","tds","unknown"]))

              precision    recall  f1-score   support

         tdp       0.94      0.92      0.93       306
         tds       0.91      0.97      0.94       276
     unknown       0.98      0.94      0.96       263

    accuracy                           0.94       845
   macro avg       0.94      0.94      0.94       845
weighted avg       0.94      0.94      0.94       845

